<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/ACLU_Project_Normalize_Data_Part_1_(Combine).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this Jupyter notebook is to combine and split the data from the various source files we were provided. The current format is extremely redundant, will cuase memory issues if handled directly in its entirety. By combining and splitting the data we will have a much easier time working with the dataset.

This notebook splits the Combined input into the following files:


*   dfMerged.csv - Includes all voting information (need to rename)
*   dfVoters.csv - Includes information with regard to the voters
*   dfOutreach.csv - Includes information based on which outreach effort(s) affected a particular voter.



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from os import listdir
from os.path import isfile, join

GoogleDriveBase = "/content/drive/My Drive/"
WorkingDirectory = "Projects/ACLU/"
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"
WorkingFiles = GoogleDriveBase + WorkingDirectory + "WorkingFiles/"
ListOfAllRequiredFiles = [WorkingDirectory + 'LetPeopleVote Data/LPVCall120210621-8755504337.xls', 
                          WorkingDirectory + 'LetPeopleVote Data/LPVMail120210621-11078584334.xls',
                          WorkingDirectory + 'LetPeopleVote Data/LPVMail220210621-13283358728.xls']
for PathToFile in ListOfAllRequiredFiles:
  path, file = os.path.split(PathToFile)

  #Break the folders out into a list
  folders = path.split(os.sep)

  RunningPath = GoogleDriveBase
  for folder in folders:
    RunningPath = RunningPath + folder + "/"
    try:
      FilesAndFolders = [f for f in listdir(RunningPath) ]
    except:
      print("The following path does not exist. Please create it before continuing.")
      print(RunningPath)

  if(os.path.exists(GoogleDriveBase + PathToFile) == False):
      print("The following file does not exist. Please create it before continuing.")
      print(GoogleDriveBase + PathToFile)

In [ ]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import csv
import string
import pandas as pd
import gc
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
AllFilesInTheSubfolder = []
for subfolder in os.walk(GoogleDriveBase + WorkingDirectory):
  #Skip the pickle folder
  if "Pickle" in subfolder[0] or "WorkingFiles" in subfolder[0]:
      continue
  for file in subfolder[2]:
    FileWithPath = subfolder[0] +"/"+ file
    match = re.search("(.*)(text|mail|phone|call|postcards)", file.lower())
    if match == None:
      raise ValueError("Could not parse: " + file + "\n Please take a look at the type of communication and ensure that it's on the list. \nThis should only happen when we add new files whose name is inconsistant with the older files.")
    CommunicationType = match.group(2).replace("_", " ").strip()
    #We have some inconsistant communication type names. So, I'm doing a little cleanup here.
    if CommunicationType == "call":
      CommunicationType = "phone"
    if CommunicationType == "postcards":
      CommunicationType = "mail"
    FileWithDetails = (FileWithPath, match.group(1).replace("_", " ").strip(), match.group(2).replace("_", " ").strip(), file)
    AllFilesInTheSubfolder.append(FileWithDetails)
    #print(FileWithDetails)

In [ ]:
#Load all the files into dataframes
columns = []
dataframesbytype = {}

for file in AllFilesInTheSubfolder:
  df = pd.read_csv(file[0], sep='\t',header=(0))
  df["ElectionType"] = file[1]
  df["CommunicationType"] = file[2]
  df["File"] = file[3]
  columns.extend(df.columns)
  if file[1] not in dataframesbytype:
    dataframesbytype[file[1]] = []
  dataframesbytype[file[1]].append(df)
columns = set(columns)
print(columns)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,28,30,35,58,59,60,61,62,63,65,69,71,73,75,77,79,82,85,87,88,89,91,92,93,95,96,97,98,99,100,101,104,106,108,109,111,113,114,115,116,117,118,119,120,121,122,123,125,126,127,129,130,131) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28,30,47,51,54,58,59,60,61,62,63,65,68,69,70,71,73,74,75,77,79,82,83,85,87,88,89,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,108,109,110,111,113,114,115,116,117,118,119,120,121,122,123,125,126,127,129,130,131) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,28,30,35,58,59

{'Zip5', 'mCity', 'Primary18Party', 'Primary01', 'Primary19Party', 'mZip5', 'AptNo', 'Home Phone', 'Primary11Party', 'DOB', 'Primary06', 'StreetNo', 'Primary10', 'Primary09', 'Primary20Party', 'Primary16Party', 'PreferredEmail', 'Special12', 'PoliceAccountability', 'Special09', 'CountyName', 'Special18', 'Primary04', 'Special13', 'Primary11', 'StreetNoHalf', 'CommunicationType', 'Voting_Nov_Gen ', 'PresPrimary16Party', 'Zip4.1', 'Primary10Party', 'HD', 'StreetPrefix', 'Sex', 'Special17', 'mAddress', 'VBM_Application', 'Special16', 'Primary17', 'Special02', 'Ballot_Application', 'DWID', 'Special08', 'General98', 'AptType', 'mZip4', 'Primary98', 'Suffix', 'FirstName', 'State.1', '2020_Biden_Support', 'Address', 'General16', 'SD', 'mState', 'Special00', 'General19', 'VotedStatus', 'Special10', 'Primary18', 'LastName', 'General06', 'Preferred Phone', 'Zip5.1', 'General09', 'Municipal13', 'Primary12', 'Special07', 'Party', 'MaritalStatus', 'General00', 'MarijuanaConviction', 'General15', 'P

In [ ]:
#Combine Dataframes of the same type
dataframesbytypejoined = {}
for dfListKey in dataframesbytype.keys():
  dataframesbytypejoined[dfListKey] = pd.concat(dataframesbytype[dfListKey])

In [ ]:
#Free up memory

del dataframesbytype
gc.collect()

50

In the following section we will be pulling out all unique voter information.

Please Note: As this data includes real world PII we will be making an effort to never show the data from this dataframe.

In [ ]:
#It looks like "Voter File VANID" is a unique key for each user, so I'm going to pull out any other person information and stick that into another dataframe. This will reduce memory footprint, which will be necessary for the pivot
#that's coming up.
dataframesbytypejoinedwithoutidvalues = {}

id_columns = ["Voter File VANID", "FirstName", "MiddleName", "LastName", "Address", "DWID", 'Zip5', 'Age', 'Primary19', 'MaritalStatus', 'mCity', 'Mass_Incarceration', 'StreetType', 'StreetSuffix',
  'mAddress', 'mState', 'mZip5', 'mZip4', 'Sex', 'City', 'State', 'Zip4', 'Suffix', 'CD', 'SD', 'HD', 'PreferredEmail', 'Preferred Phone', 'Cell Phone', 'CountyName', 'DOB', 'DateReg', 'Home Phone', 
  'EthnicCatalistName', 'Party', 'PersonalEmail', 'RaceName', 'StreetNo', 'StreetNoHalf', 'StreetPrefix', 'StreetName', 'AptType', 'AptNo', 'City.1', 'State.1', 'Zip5.1', 'Zip4.1', 
  '2020_Biden_Support', 'Voting_Aug_Prim', 'PoliceAccountability', 'VBM_Application', 'VotedStatus', 'MarijuanaConviction', '2020_Biden_Support', 'Absentee_Voting', 'Voting_Nov_Gen']
primary_key = ["Voter File VANID"]
dfVoters = None
for dfListKey in dataframesbytypejoined.keys():
  df = dataframesbytypejoined[dfListKey]
  ColumnsToRemove = list(set(id_columns) - set(primary_key))
  if dfVoters is None:
    dfVoters = df[id_columns]
    dataframesbytypejoinedwithoutidvalues[dfListKey] = df.drop(columns=ColumnsToRemove)
    continue
  dfVoters.append(df[id_columns])
  dataframesbytypejoinedwithoutidvalues[dfListKey] = df.drop(columns=ColumnsToRemove)
dfVoters = dfVoters.drop_duplicates()
dfVoters = dfVoters.sort_values(by=['FirstName', 'LastName'])

,Voter File VANID,FirstName,MiddleName,LastName,Address,DWID,Zip5,Age,Primary19,MaritalStatus,mCity,Mass_Incarceration,StreetType,StreetSuffix,mAddress,mState,mZip5,mZip4,Sex,City,State,Zip4,Suffix,CD,SD,HD,PreferredEmail,Preferred Phone,Cell Phone,CountyName,DOB,DateReg,Home Phone,EthnicCatalistName,Party,PersonalEmail,RaceName,StreetNo,StreetNoHalf,StreetPrefix,StreetName,AptType,AptNo,City.1,State.1,Zip5.1,Zip4.1
43680,2724858,A,NaN,Abel,2420 E Provincial House Dr,20054367,48910,61.0,NaN,S,Lansing,NaN,Dr,NaN,2420 E Provincial House Dr,MI,48910.0,4880.0,M,Lansing,MI,4880.0,NaN,8.0,23.0,68.0,NaN,"=""9198050512""","=""9198050512""",Ingham,01/01/1960,04/12/1995,NaN,NaN,U,NaN,Caucasian,2420.0,NaN,E,Provincial House,NaN,NaN,Lansing,MI,"=""48910""","=""4880"""
69072,4238109,A,J,Adams,509 Soule Blvd,21603224,48103,69.0,NaN,M,Ann Arbor,NaN,Blvd,NaN,509 Soule Blvd,MI,48103.0,4682.0,M,Ann Arbor,MI,4682.0,NaN,12.0,18.0,53.0,NaN,"=""7349459531""","=""7349459531""",Washtenaw,09/10/1951,10/12/1982,"=""7346655529""",Uncoded,U,NaN,Caucasian,509.0,NaN,NaN,Soule,NaN,NaN,Ann Arbor,MI,"=""48103""","=""4682"""
202934,17182356,A,Jay,Anthony,1522 10th Ave Apt 8,926117401,48060,39.0,NaN,NaN,Port Huron,NaN,Ave,NaN,1522 10th Ave Apt 8,MI,48060.0,3366.0,M,Port Huron,MI,3366.0,NaN,10.0,25.0,83.0,NaN,"=""8106628784""","=""8106628784""",St. Clair,01/01/1982,05/22/2015,NaN,NaN,U,NaN,Caucasian,1522.0,NaN,NaN,10th,Apt,8,Port Huron,MI,"=""48060""","=""3366"""
99402,18856761,A,Spencer,Barefield,19550 Argyle Cres,17462192,48203,68.0,NaN,NaN,Detroit,NaN,Cres,NaN,19550 Argyle Cres,MI,48203.0,1400.0,M,Detroit,MI,1400.0,NaN,14.0,4.0,3.0,NaN,NaN,NaN,Wayne,01/01/1953,10/09/2001,NaN,Uncoded,NaN,NaN,Black,19550.0,NaN,NaN,Argyle,NaN,NaN,Detroit,MI,"=""48203""","=""1400"""
162186,13750168,A,J,Campbell,13590 Garfield,401521472,48239,60.0,NaN,S,Redford,NaN,NaN,NaN,13590 Garfield,MI,48239.0,4513.0,M,Redford,MI,4513.0,NaN,13.0,5.0,10.0,NaN,"=""3134245741""","=""3134245741""",Wayne,01/01/1961,04/07/2009,"=""3135377650""",NaN,U,NaN,Caucasian,13590.0,NaN,NaN,Garfield,NaN,NaN,Redford,MI,"=""48239""","=""4513"""


In [ ]:
dfVoters.reset_index(inplace=True)
dfVoters.to_csv(WorkingFiles + "dfVoters.csv", encoding='utf-8')

del dfVoters
gc.collect()

<function gc.collect>

In [ ]:
del dataframesbytypejoined
gc.collect()

50

In [ ]:
dfMerged = pd.concat(dataframesbytypejoinedwithoutidvalues.values())

del dataframesbytypejoinedwithoutidvalues
gc.collect()

0

In [ ]:
#Pull out the file information, this indicates which outreach efforts they were included in.
dfOutreach = dfMerged[["Voter File VANID", "ElectionType", "CommunicationType", "File"]]
dfOutreach.reset_index(inplace=True)
dfOutreach.to_csv(WorkingFiles + "dfOutreach.csv", encoding='utf-8')

del dfOutreach
gc.collect()

dfMerged = dfMerged.drop(columns=["ElectionType", "CommunicationType", "File"])

In [ ]:
print(dfMerged.columns)

Index(['Voter File VANID', 'StateFileID', 'General20', 'General19',
       'General18', 'General17', 'General16', 'General15', 'General14',
       'General12', 'General11', 'General10', 'General09', 'General08',
       'General06', 'General04', 'General02', 'General00', 'General98',
       'Municipal13', 'PresPrimary20', 'PresPrimary20Party', 'PresPrimary16',
       'PresPrimary16Party', 'PresPrimary12', 'PresPrimary12Party',
       'PresPrimary08', 'PresPrimary08Party', 'Primary20', 'Primary20Party',
       'Primary19Party', 'Primary18', 'Primary18Party', 'Primary17',
       'Primary17Party', 'Primary16', 'Primary16Party', 'Primary15',
       'Primary15Party', 'Primary14', 'Primary14Party', 'Primary12',
       'Primary12Party', 'Primary11', 'Primary11Party', 'Primary10',
       'Primary10Party', 'Primary09', 'Primary09Party', 'Primary08',
       'Primary08Party', 'Primary06', 'Primary04', 'Primary02', 'Primary01',
       'Primary00', 'Primary98', 'Special20', 'Special19', 'Special18',

In [ ]:

dfMerged.reset_index(inplace=True)


Finally, we will save the data to disk for use by the next Jupyter notebook.

[Merge Data](https://colab.research.google.com/drive/1epgGr9YFq1l3BMvDLApkWXnZ9TjCcJCo?usp=sharing)

In [ ]:

dfMerged.to_csv(WorkingFiles + "dfMerged.csv", encoding='utf-8')
